In [2]:
%load_ext autoreload

# Notebook Initialization

In [3]:
%autoreload
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['figure.figsize'] = [10,10]

import numpy as np
import pickle

import darkhistory.physics as phys
import darkhistory.spec.transferfunction as tf

# Import ICS Tables

In [4]:
nonrel_ICS_table = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ICS_nonrel.raw","rb"))
nonrel_ICS_table = np.array(nonrel_ICS_table, dtype='float64')
rel_ICS_table = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ICS_rel.raw","rb"))
rel_ICS_table = np.array(rel_ICS_table, dtype='float64')
engloss_table = pickle.load(open("/Users/hongwan/Dropbox (MIT)/Photon Deposition/ICS_englossspec.raw","rb"))
engloss_table = np.array(engloss_table, dtype='float64')